In [1]:
# import packages
import os
from PIL import Image
import numpy as np

# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# print(tf.__version__)
OPENSLIDE_PATH = r'C:\Program Files\openslide-win64-20230414\bin'

os.environ['PATH'] = OPENSLIDE_PATH + ";" + os.environ['PATH']

if hasattr(os, 'add_dll_directory'):
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

from openslide import OpenSlide


In [2]:
# set output path for 10x and 2x tif images - create directory if they don't exist
WSI_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files'
path_10x = os.path.join(WSI_path, '10x')
path_2x = os.path.join(WSI_path, '2x')  # actually 2.5
if not os.path.exists(path_10x):
    os.mkdir(path_10x)
if not os.path.exists(path_2x):
    os.mkdir(path_2x)

In [4]:

# find missing images -> code will continue with the missing ones and skip the ones that are existed
wsis = [_ for _ in os.listdir(WSI_path) if _.endswith('svs')]
wsis2 = [os.path.splitext(_)[0] for _ in wsis]
tenx = [_ for _ in os.listdir(path_10x) if _.endswith('tif')]
tenx2 = [os.path.splitext(_)[0] for _ in tenx]
missingimages = [_ for _ in wsis2 if _ not in tenx2]

In [5]:

for idx, missingimage in enumerate(missingimages):
    print(idx, '/', len(missingimages), ' processing: ', missingimage)
    wsi = OpenSlide(os.path.join(WSI_path, missingimage + '.svs'))

    rsf = 1  # 1 micorn/pixel = 10x
    svs_img = wsi.read_region(location=(0, 0), level=0, size=wsi.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf / float(wsi.properties['openslide.mpp-x'])  #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf / float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0] / resize_factorx)), int(np.ceil(wsi.dimensions[1] / resize_factory))])
    svs_img10x = svs_img.resize(resize_dimension, resample=Image.NEAREST)
    svs_img10x.save(os.path.join(path_10x, missingimage + '.tif'), resolution=1, resolution_unit=1, quality=100, compression=None)

    rsf = 4
    resize_factorx = rsf / float(wsi.properties['openslide.mpp-x'])  #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf / float(wsi.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(wsi.dimensions[0] / resize_factorx)), int(np.ceil(wsi.dimensions[1] / resize_factory))])
    svs_img2_5x = svs_img.resize(resize_dimension, resample=Image.NEAREST)
    svs_img2_5x.save(os.path.join(path_2x, missingimage + '.tif'), resolution=2_5, resolution_unit=1, quality=100, compression=None)

0 / 398  processing:  6521-002


MemoryError: 